In [ ]:
import pickle
import keras
from keras.optimizers.legacy import Adam as LegacyAdam
import numpy as np
from src.utils.preprocessing import load_split_data
import json
from src.utils.data_io import save_data
from src.utils.train_cnn import predict_and_save_timestamped
from src.utils.utilities import *

In [ ]:
path_to_predictions = "../data/cnn_predictions/non-majority/un-timestamped/full/"
path_to_start_time_json = "../data/dataset-info-json/signal_start_times.json"
path_to_save_timestamped = "../data/cnn_predictions/non-majority/timestamped/in_sessions/"

In [ ]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [ ]:
with open("../data/dataset-info-json/signal_start_times.json", "r") as f:
    session_start_time_and_length = json.load(f)
session_length = [value[1] for value in session_start_time_and_length.values()]

In [ ]:
def save_predictions_untimestamped():   
    for test_subject_id in subject_to_indices.keys():
        model = keras.models.load_model(f"../models/full_loso/majority_label/model_{test_subject_id}.keras", compile=False)
        optimizer = LegacyAdam(learning_rate=1e-3)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
        _, _, test_data, test_labels = load_split_data(test_subject_id, subject_to_indices)
        predictions = []
        for window in test_data:
            window_reshaped = window.reshape(1,20,6)
            prediction = model.predict(window_reshaped)
            predictions.append(prediction)
        predictions = np.array(predictions)
        predictions = predictions.squeeze(axis=1)
        save_data(predictions, "../data/cnn_predictions/non-majority/un_timestamped/full", f"prediction_{test_subject_id}")
    return

In [ ]:
def save_predictions_timestamped_in_sessions():
    for subject, session_indices in subject_to_indices.items():  
        for session in session_indices:
            print(f"Subject: {subject}\n\tSession:{session}")
            with open(f"../data/cnn_predictions/non-majority/un-timestamped/in_sessions/prediction_{subject}_{session}.pkl", "rb") as f:
                prediction = pickle.load(f)
            start_time = load_start_time(start_time_json_path, session)
            print(f"\t\tStart time: {start_time}")
            timestamped_predictions = append_timestamps_to_predictions(prediction, start_time)
            save_data(timestamped_predictions, path_to_save_timestamped, f"prediction_{subject}_{session}")

In [ ]:
save_predictions_untimestamped()

In [ ]:
sessioned_predictions = split_predictions_to_sessions(path_to_predictions, subject_to_indices, session_length)

In [ ]:
counter = 0
for subject, sessions in subject_to_indices.items():
    for session in sessions:
        save_data(predictions_in_sessions[counter], path_to_save, f"prediction_{subject}_{session}")
        counter += 1

In [ ]:
save_predictions_timestamped_in_sessions()